In [3]:
import os
from typing import List, Optional, Dict, Tuple
from tqdm import tqdm
import json
from collections import Counter
import gensim
from indexingcode.utils.Preprocessor import Preprocessor
import nltk
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import ijson

In [11]:
preprocessor = Preprocessor(gensim.utils.tokenize, nltk.PorterStemmer())

def get_text(node):
    try:
        return node.text
    except AttributeError:
        return None


file_path: str = '/Volumes/SanDisk/BioAsq2021/allMeSH_2021.json'
write_file_dir: str = '/Volumes/SanDisk/med_data_stem'

tok_preprocessor = Preprocessor(gensim.utils.tokenize, None)
NN = 100000
mh_set = set()
MAX_MHS = 20
with open(file_path, encoding='ISO-8859-1') as bioasq_file:
    client_id = 1
    for article in tqdm(ijson.items(bioasq_file, 'articles.item'), total=15559157):
        if client_id > NN:
            break
        text: str = article['abstractText']
        sentences: List[str] = text.split('.')
        sentences_tokens: List[List[str]] = [preprocessor.preprocess_text(s) for s in sentences]
        for mh in article['meshMajor']:
            if mh not in mh_set and len(mh_set) > MAX_MHS:
                continue
            mh_set.add(mh)
            with open(os.path.join(write_file_dir, f"{mh}.csv"), 'a') as write_f:
                tid = 1
                for items in sentences_tokens:
                    write_f.write(f"{str(client_id)} {str(tid)} {' '.join([str(_) for _ in items])}\n")
                    tid += 1
        client_id += 1

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wojciechsitek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wojciechsitek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
  1%|          | 100000/15559157 [12:00<30:56:51, 138.76it/s]


In [12]:
with open('/Volumes/SanDisk/med_vocabulary_stem.csv', 'w') as f:
    for token, num in tok_preprocessor.vocabulary.items():
        f.write(f"{num} {token}\n")

In [13]:
with open('/Volumes/SanDisk/med_vocabulary_stem.csv', 'w') as f:
    for token, num in preprocessor.vocabulary.items():
        f.write(f"{num} {token}\n")